In [1]:
!pip install transformers==4.8.2

     |████████████████████████████████| 2.5 MB 25.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 53.8 MB/s eta 0:00:01
     |████████████████████████████████| 748 kB 52.9 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 10.8 MB/s eta 0:00:01


In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments


# Dataset tokenization and encoding in progress
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset


# Dynamically padding to the maximum length if the inputs are not all the same length
def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator


def train(train_file_path, model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs):

    # Load tokenizer, dataset, collator, model
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    # Setting training parameters
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
    )

    # Setting Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    # Training
    trainer.train()
    # Save the tokenizer and the trained model
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

In [3]:
train_file_path = "./preprocessed.txt"
model_name = 'gpt2'
output_dir = './result'
overwrite_output_dir = False
per_device_train_batch_size = 32
num_train_epochs = 5.0

In [4]:
train(train_file_path, model_name,
      output_dir,
      overwrite_output_dir,
      per_device_train_batch_size,
      num_train_epochs)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 1221
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 195


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./result
Configuration saved in ./result/config.json
Model weights saved in ./result/pytorch_model.bin
tokenizer config file saved in ./result/tokenizer_config.json
Special tokens file saved in ./result/special_tokens_map.json
